In [ ]:
import os
import copy
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.parameter import Parameter
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from torch.nn import Mish
import wandb

In [ ]:
# Load data
listing_path = 'listings_clean.csv'
listing_data = pd.read_csv(listing_path)
print(listing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7391 entries, 0 to 7390
Data columns (total 80 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   host_is_superhost                                                              7391 non-null   float64
 1   host_listings_count                                                            7391 non-null   float64
 2   host_identity_verified                                                         7391 non-null   int64  
 3   latitude                                                                       7391 non-null   float64
 4   longitude                                                                      7391 non-null   float64
 5   accommodates                                                                   7391 non-null   float64
 6   bedrooms                

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


X = listing_data.drop(columns=['price']).values
y = listing_data['price'].values

# Feature selection
rf = RandomForestRegressor(random_state=42, n_estimators=100)
rf.fit(X, y)
sfm = SelectFromModel(rf, threshold=0.002, prefit=True)
X_important = sfm.transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_important, y, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

# Create datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_features = X_train.shape[1]
output_dim = 1

# Define neural network
class Net(nn.Module):
    def __init__(self, input_features, dropout=0.6):
        super(Net, self).__init__()
        self.x_dim = input_features
        self.z_dim = 256

        self.main_net = nn.Sequential(
            nn.Linear(self.x_dim, self.z_dim),
            nn.BatchNorm1d(self.z_dim),
            Mish(),
            nn.Dropout(dropout),
            nn.Linear(self.z_dim, 256),
            nn.BatchNorm1d(256),
            Mish(),
            nn.Dropout(dropout),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            Mish(),
            nn.Dropout(dropout),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            Mish(),
            nn.Dropout(dropout),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            Mish(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            Mish(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            Mish(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            Mish(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            Mish(),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.main_net(x)

In [ ]:
# Define Train function
def train_epoch(model, device, train_loader, optimizer):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()

# Define Test function
def test_epoch(model, device, test_loader):
    model.eval()
    test_loss = 0
    all_targets, all_outputs = [], []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item()
            all_targets.append(target.cpu().numpy())
            all_outputs.append(output.cpu().numpy())

    test_loss /= len(test_loader.dataset)
    rmse = np.sqrt(test_loss)
    all_targets = np.concatenate(all_targets)
    all_outputs = np.concatenate(all_outputs)
    r2 = r2_score(all_targets, all_outputs)
    return rmse, r2

In [ ]:
# Define wandb config
wandb.login(key="xxx") # Change key to your own key if you want to log to wandb
os.environ["WANDB_SILENT"] = "true"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: francescogiannuzzo2002-fg (mldlfragian). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Define hyperparameters
sweep_config = {
    "method": "grid",
    "metric": {"name": "rmse", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"values": [0.001, 0.01]},
        "weight_decay": {"values": [1e-8, 1e-6, 1e-4]},
        "dropout": {"values": [0.4, 0.5, 0.6]},
        "batch_size": {"values": [64, 128, 256]}
    },
}

In [ ]:
# Initialize wandb sweep
sweep_id = wandb.sweep(sweep_config, project="grid_search_tuning_MALIS_f")
results = []

# Define train sweep function
def train_sweep(config=None):
        wandb.init(project="grid_search_tuning_MALIS_f", settings=wandb.Settings(silent="true"))
        config = wandb.config

        train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False)

        model = Net(input_features=X_train.shape[1], dropout=config.dropout).to(device)
        optimizer = optim.RMSprop(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, alpha=0.9,eps=1e-16)
    
        for epoch in range(1, 501):
            train_epoch(model, device, train_loader, optimizer)

        rmse, r2 = test_epoch(model, device, test_loader)
        wandb.log({"rmse": rmse, "r2": r2})

        results.append({
            "rmse": rmse,
            "r2": r2,
            "config": {
                "learning_rate": config.learning_rate,
                "weight_decay": config.weight_decay,
                "dropout": config.dropout,
                "batch_size": config.batch_size,
            },
        })

# Run wandb agent
wandb.agent(sweep_id, function=train_sweep)
wandb.finish()

wandb: Agent Starting Run: nydnvbp4 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.4
wandb: 	learning_rate: 0.001
wandb: 	weight_decay: 1e-08


In [ ]:
# Print top 5 results
results = sorted(results, key=lambda x: x["rmse"])
print("Top 5:")
for i, res in enumerate(results[:5]):
    print(f"Rank {i+1}: RMSE={res['rmse']}, R2={res['r2']}, Config={res['config']}")

Top 5:
Rank 1: RMSE=96.87442882991559, R2=0.7587296022515624, Config={'learning_rate': 0.001, 'weight_decay': 1e-06, 'dropout': 0.4, 'batch_size': 128}
Rank 2: RMSE=97.21548287115348, R2=0.7570277912547838, Config={'learning_rate': 0.001, 'weight_decay': 1e-08, 'dropout': 0.6, 'batch_size': 128}
Rank 3: RMSE=97.62904980560941, R2=0.7549561265019948, Config={'learning_rate': 0.001, 'weight_decay': 1e-08, 'dropout': 0.6, 'batch_size': 256}
Rank 4: RMSE=98.05535479059881, R2=0.7528114386440358, Config={'learning_rate': 0.001, 'weight_decay': 1e-06, 'dropout': 0.5, 'batch_size': 256}
Rank 5: RMSE=98.17726173149939, R2=0.7521964269715234, Config={'learning_rate': 0.001, 'weight_decay': 0.0001, 'dropout': 0.5, 'batch_size': 128}


In [ ]:
# Retrain top 5 configurations for 1500 epochs
best_results = []

for i, best_config in enumerate(results[:5]):
    # Initialize wandb
    wandb.init(project="network_best_regression_f", name=f"config_{i+1}", settings=wandb.Settings(silent="true"))
    print(f"Retraining configuration {i + 1} for 1500 epochs: {best_config['config']}")
    config = best_config["config"]
    # Create data loaders
    train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_data, batch_size=config["batch_size"], shuffle=False)
    # Define model
    model = Net(input_features=X_train.shape[1], dropout=config["dropout"]).to(device)
    # Define optimizer
    optimizer = optim.RMSprop(model.parameters(),lr=config["learning_rate"],weight_decay=config["weight_decay"],alpha=0.9,eps=1e-16)
    best_rmse = float("inf")

    # Train for each epoch and log results
    for epoch in range(1, 1501):
        train_epoch(model, device, train_loader, optimizer)
        rmse, r2 = test_epoch(model, device, train_loader)
        if epoch % 50 == 0: wandb.log({"epoch": epoch, "train_rmse": rmse, "train_r2": r2})
        rmse, r2 = test_epoch(model, device, test_loader)
        if epoch % 50 == 0: wandb.log({"epoch": epoch, "test_rmse": rmse, "test_r2": r2})
        if rmse < best_rmse:
            best_rmse = rmse
            best_r2 = r2

        if epoch % 500 == 0:
            print(f"Epoch {epoch}: RMSE={rmse:.4f}, Best RMSE={best_rmse:.4f}")

    best_results.append({
        "best_rmse": best_rmse,
        "best_r2": best_r2,
        "config": config,
    })
    wandb.finish()
    
# Print best results
best_results = sorted(best_results, key=lambda x: x["best_rmse"])
print("\nBest configuration after retraining:")
print(f"RMSE={best_results[0]['best_rmse']}, R2={best_results[0]['best_r2']}, Config={best_results[0]['config']}")

Retraining configuration 1 for 1500 epochs: {'learning_rate': 0.001, 'weight_decay': 1e-06, 'dropout': 0.4, 'batch_size': 128}
Epoch 500: RMSE=101.1443, Best RMSE=96.0929
Epoch 1000: RMSE=101.4612, Best RMSE=96.0929
Epoch 1500: RMSE=103.2186, Best RMSE=96.0929


Retraining configuration 2 for 1500 epochs: {'learning_rate': 0.001, 'weight_decay': 1e-08, 'dropout': 0.6, 'batch_size': 128}
Epoch 500: RMSE=96.8804, Best RMSE=95.8535
Epoch 1000: RMSE=96.7926, Best RMSE=94.9880
Epoch 1500: RMSE=97.3618, Best RMSE=94.9880


Retraining configuration 3 for 1500 epochs: {'learning_rate': 0.001, 'weight_decay': 1e-08, 'dropout': 0.6, 'batch_size': 256}
Epoch 500: RMSE=97.2568, Best RMSE=95.5222
Epoch 1000: RMSE=99.4850, Best RMSE=95.5222
Epoch 1500: RMSE=101.1628, Best RMSE=95.5222


Retraining configuration 4 for 1500 epochs: {'learning_rate': 0.001, 'weight_decay': 1e-06, 'dropout': 0.5, 'batch_size': 256}
Epoch 500: RMSE=100.4953, Best RMSE=95.0686
Epoch 1000: RMSE=101.9900, Best RMSE=94.0233
Epoch 1500: RMSE=98.5272, Best RMSE=93.3325


Retraining configuration 5 for 1500 epochs: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'dropout': 0.5, 'batch_size': 128}
Epoch 500: RMSE=101.2396, Best RMSE=95.2558
Epoch 1000: RMSE=97.7611, Best RMSE=94.6949
Epoch 1500: RMSE=96.8896, Best RMSE=94.3560



Best configuration after retraining:
RMSE=93.33247956119693, R2=0.7760498628800443, Config={'learning_rate': 0.001, 'weight_decay': 1e-06, 'dropout': 0.5, 'batch_size': 256}
